In [1]:
import cv2 as cv
from cv2 import VideoCapture
import numpy as np
from typing import Callable, Tuple

In [2]:
cap = VideoCapture()


class ViewController:
    @staticmethod
    def coord_midpoint_to_corner(mid_y: int, mid_x: int, h: int, w: int) -> Tuple[int, int, int, int]:
        cor_y = mid_y - h // 2
        cor_x = mid_x - w // 2
        return cor_y, cor_x, h, w

    @staticmethod
    def coord_corner_to_midpoint(cor_y: int, cor_x: int, h: int, w: int) -> Tuple[int, int, int, int]:
        mid_y = cor_y + h // 2
        mid_x = cor_x + w // 2
        return mid_y, mid_x, h, w

    def __init__(
        self,
        video_stream: VideoCapture,
        camera_size: Tuple[int, int] = (201, 201),
        micro_size: Tuple[int, int] = (41, 41),
        init_position: Tuple[int, int] = (0, 0),
        padding_value=0,
    ) -> None:
        self._video_stream = video_stream
        self._camera_size = camera_size
        self._micro_size = micro_size
        self._padding_value = padding_value

        frame_width = int(video_stream.get(cv.CAP_PROP_FRAME_WIDTH))
        frame_height = int(video_stream.get(cv.CAP_PROP_FRAME_HEIGHT))
        self._frame_size = (frame_height, frame_width)

        self._position = (0, 0)
        self.set_position(*init_position)

        self._frame = None
        res = self.next_frame()

        if res is False:
            raise Exception("couldn't read first frame")

    def skip_frames(self, count: int) -> bool:
        for _ in range(count):
            res, _ = self._video_stream.read()
            if res is False:
                break
        return self.next_frame()

    def next_frame(self) -> bool:
        ret, frame = self._video_stream.read()
        if ret == False:
            self._frame = None
            return False

        cam_height, cam_width = self.get_camera_size()
        pad_height, pad_width = cam_height // 2, cam_width // 2

        self._frame = cv.copyMakeBorder(
            src=frame,
            top=pad_height,
            bottom=pad_height,
            left=pad_width,
            right=pad_width,
            borderType=cv.BORDER_CONSTANT,
            value=self._padding_value,
        )

        return True

    def get_frame_size(self) -> Tuple[int, int]:
        return self._frame_size

    def get_camera_size(self):
        return self._camera_size

    def get_micro_size(self):
        return self._micro_size

    def get_midpoint_position(self) -> Tuple[int, int]:
        return self._position

    def set_position(self, y: int, x: int):
        assert y >= 0 and y < self._frame_size[0]
        assert x >= 0 and x < self._frame_size[1]
        self._position = (y, x)

    def move_position(self, dy: int, dx: int):
        self.set_position(self._position[0] + dy, self._position[1] + dx)

    def get_camera_view(self) -> np.ndarray:
        y, x, h, w = ViewController.coord_midpoint_to_corner(*self._position, *self.get_camera_size())
        y, x = y + h // 2, x + w // 2
        slice = self._frame[y : y + h, x : x + w]
        return slice

    def get_micro_view(self) -> np.ndarray:
        y, x, h, w = ViewController.coord_midpoint_to_corner(*self._position, *self.get_micro_size())
        y, x = y + h // 2, x + w // 2
        slice = self._frame[y : y + h, x : x + w]
        return slice

    def visualize(self):
        cam_y, cam_x, cam_h, cam_w = ViewController.coord_midpoint_to_corner(*self._position, *self.get_camera_size())
        cam_y, cam_x = cam_y + cam_h // 2, cam_x + cam_w // 2

        mic_y, mic_x, mic_h, mic_w = ViewController.coord_midpoint_to_corner(*self._position, *self.get_micro_size())
        mic_y, mic_x = mic_y + cam_h // 2, mic_x + cam_w // 2

        image = self._frame.copy()
        cv.rectangle(image, (cam_x, cam_y), (cam_x + cam_w, cam_y + cam_h), (0, 0, 255), 5)
        cv.rectangle(image, (mic_x, mic_y), (mic_x + mic_w, mic_y + mic_h), (0, 255, 0), 5)

        cv.imshow("img", image)
        cv.waitKey(1)

    def close_windows(self):
        cv.destroyAllWindows()

In [3]:
cap = VideoCapture("worms.avi")

sim = ViewController(cap)

while sim.next_frame():
    sim.visualize()
    sim.move_position(50, 50)

sim.close_windows()

KeyboardInterrupt: 

In [ ]:
cap = VideoCapture("worms.avi")

_, frame = cap.read()

print(frame.shape)

#cv.imshow("img", frame)

In [ ]:
res, _ = cap.read()
print(res)

In [ ]:
cv.destroyAllWindows()